In [1]:


import sparknlp
 
spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/12/06 13:57:36 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/12/06 13:57:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-407c6019-a4b2-4974-aa2e-6534033656c2;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.project

22/12/06 13:57:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [3]:
import pyspark.sql.functions as F

#training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("pos"),
#        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


In [4]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [5]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]


In [6]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-12-06 13:57:48.093074: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 13:57:48.093091: I external/org_tensorflow/tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-06 13:57:48.568719: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 13:57:48.569415: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open 

Training started - total epochs: 500 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 24 - training examples: 246
Epoch 1/500 started, lr: 0.003, dataset size: 246
Epoch 1/500 - 0.83s - loss: 39.522293 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 2	 0	 17	 1.0	 0.10526316	 0.1904762
Loc	 0	 0	 18	 0.0	 0.0	 0.0
Act	 22	 6	 19	 0.78571427	 0.5365854	 0.6376811
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 73	 47	 7	 0.60833335	 0.9125	 0.73
Obj	 15	 12	 0	 0.5555556	 1.0	 0.71428573
Pos	 0	 0	 3	 0.0	 0.0	 0.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 129 fp: 65 fn: 65 labels: 8
Macro-average	 prec: 0.4937004, rec: 0.44429356, f1: 0.46769577
Micro-average	 prec: 0.66494846, rec: 0.66494846, f1: 0.66494846
Epoch 2/500 started, lr: 0.0029850747, dataset size: 246
Epoch 2/500 - 0.19s - loss: 26.993752 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.0

Epoch 13/500 - 0.16s - loss: 0.5857791 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.9625	 0.9808917
Obj	 15	 1	 0	 0.9375	 1.0	 0.9677419
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 191 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.98637354, rec: 0.9953125, f1: 0.99082285
Micro-average	 prec: 0.9845361, rec: 0.9845361, f1: 0.9845361
Epoch 14/500 started, lr: 0.0028169013, dataset size: 246
Epoch 14/500 - 0.16s - loss: 0.41377267 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 76	 0	 4	 1.0	 0.95	 0.97

Epoch 26/500 - 0.16s - loss: 0.095789716 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 76	 0	 4	 1.0	 0.95	 0.9743589
Obj	 15	 2	 0	 0.88235295	 1.0	 0.9375
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 190 fp: 4 fn: 4 labels: 8
Macro-average	 prec: 0.97948015, rec: 0.99375, f1: 0.9865635
Micro-average	 prec: 0.97938144, rec: 0.97938144, f1: 0.97938144
Epoch 27/500 started, lr: 0.0026548672, dataset size: 246
Epoch 27/500 - 0.16s - loss: 0.13297147 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 76	 0	 4	 1.0	 0.95	 0.9

Epoch 40/500 - 0.15s - loss: 0.03074748 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.9625	 0.9808917
Obj	 15	 1	 0	 0.9375	 1.0	 0.9677419
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 191 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.98637354, rec: 0.9953125, f1: 0.99082285
Micro-average	 prec: 0.9845361, rec: 0.9845361, f1: 0.9845361
Epoch 41/500 started, lr: 0.0025, dataset size: 246
Epoch 41/500 - 0.15s - loss: 0.06887619 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.9625	 0.98089

...

Epoch 460/500 - 0.16s - loss: 1.5164506E-4 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.9625	 0.9808917
Obj	 15	 1	 0	 0.9375	 1.0	 0.9677419
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 191 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.98637354, rec: 0.9953125, f1: 0.99082285
Micro-average	 prec: 0.9845361, rec: 0.9845361, f1: 0.9845361
Epoch 461/500 started, lr: 9.090909E-4, dataset size: 246
Epoch 461/500 - 0.16s - loss: 9.4482844E-4 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.9

Epoch 474/500 - 0.16s - loss: 8.3107565E-4 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.9625	 0.9808917
Obj	 15	 1	 0	 0.9375	 1.0	 0.9677419
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 191 fp: 3 fn: 3 labels: 8
Macro-average	 prec: 0.98637354, rec: 0.9953125, f1: 0.99082285
Micro-average	 prec: 0.9845361, rec: 0.9845361, f1: 0.9845361
Epoch 475/500 started, lr: 8.9020777E-4, dataset size: 246
Epoch 475/500 - 0.16s - loss: 6.67209E-4 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 0	 0	 1.0	 1.0	 1.0
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 41	 2	 0	 0.95348835	 1.0	 0.97619045
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 77	 0	 3	 1.0	 0.96

Epoch 488/500 - 0.16s - loss: 0.012447474 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 3	 0	 0.8636364	 1.0	 0.9268293
Loc	 18	 1	 0	 0.94736844	 1.0	 0.972973
Act	 40	 2	 1	 0.95238096	 0.9756098	 0.9638555
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 74	 0	 6	 1.0	 0.925	 0.961039
Obj	 15	 1	 0	 0.9375	 1.0	 0.9677419
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 17	 0	 0	 1.0	 1.0	 1.0
tp: 187 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.9626107, rec: 0.98757625, f1: 0.9749337
Micro-average	 prec: 0.96391755, rec: 0.96391755, f1: 0.96391755
Epoch 489/500 started, lr: 8.72093E-4, dataset size: 246
Epoch 489/500 - 0.16s - loss: 0.10973539 - batches: 9
Quality on validation dataset (20.0%), validation examples = 49
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 19	 3	 0	 0.8636364	 1.0	 0.9268293
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 40	 2	 1	 0.95238096	 0.9756098	 0.9638555
Neg	 1	

CPU times: user 287 ms, sys: 90.9 ms, total: 378 ms
Wall time: 1min 30s


2022-12-06 13:59:16.228110: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [7]:

#predictions = ner_model.transform(testing_data) 
#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#           .select(F.expr("cols['0']").alias("token"),
#                   F.expr("cols['1']").alias("ground_truth"),
#                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [8]:
from sklearn.metrics import classification_report
 
#preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#                      .select(F.expr("cols['0']").alias("token"),
#                              F.expr("cols['1']").alias("ground_truth"),
#                              F.expr("cols['2']").alias("prediction")).toPandas()
 
#print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

In [9]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [10]:
text_list = [
    """Navagate to kitchen"""
]


In [11]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


2022-12-06 13:59:17.076860: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [12]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_chunk': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'embeddings': ['navigate', 'to', 'kitchen']}
['Act', '0', 'Loc']


In [13]:
light.annotate("Navigate to kitchen")


{'document': ['Navigate to kitchen'],
 'ner_chunk': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'embeddings': ['Navigate', 'to', 'kitchen']}

In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

Enter Testing Text
cabinet
['Obj']


In [ ]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [ ]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




